# Scraper Review

## Take Aways
* Row content may not be constant across scrapers - as in instead of a STATIC series of fields attempting to be filled , some scrapers only collect a few things and they may be in a differnt order (ADB vs. Panel for instance)
* Current ADB scraper (may 24th) is broken - hangs on one project. Website looks to have a table per year (2018, 2017) and scraper is getting stuck repeating last row of the 2018 table. 

## EIB Scraper

Currently NOT working. 

In [1]:

def scrape():
    with open('eib_scraped.csv', 'wb') as file:
        writer = unicodecsv.writer(file)
        header = [
            'IAM',
            'Year',
            'Country',
            'Project',
            'ID',
            'IAM ID',
            'Filer(s)',
            'Environmental Category',
            'Project Company',
            'Project Number',
            'Related Project Number',
            'Project Type',
            'Financial Institution',
            'Project Loan Amount',
            'Sector',
            'Issues',
            'Complaint Status',
            'Filing Date',
            'Registration Start Date',
            'Registration End Date',
            'Eligibility Start Date',
            'Eligibility End Date',
            'Dispute Resolution Start Date',
            'Dispute Resolution End Date',
            'Compliance Review Start Date',
            'Compliance Review End Date',
            'Monitoring Start Date',
            'Monitoring End Date',
            'Date Closed',
            'Documents',
            'Hyperlink',
            'Project Date',
            'Project Status',
            'Project Description',
        ]
        writer.writerow(header)
        driver = webdriver.Chrome()
        eib_scrape(driver, writer)
        time.sleep(2)
        driver.quit()

def eib_scrape(driver, writer):
    driver.get('http://www.eib.org/about/accountability/complaints/cases/index.htm')
    time.sleep(2)
    select_all = driver.find_element_by_xpath('//*[@id="consultationsList"]/div/div[3]/div[1]/select/option[5]')
    select_all.click()
    time.sleep(2)
#     button = driver.find_element_by_xpath('//*[@id="helpUsWebsite"]/div/div[2]/a[1]').click()
#     time.sleep(1)
    button = driver.find_element_by_xpath('//*[@id="footer"]/div[3]/div[2]/div/p/button').click()
    time.sleep(1)
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
    time.sleep(1)
    rows = driver.find_elements_by_xpath('//*[@id="consultationsList"]/div/table/tbody/tr')
    row_range = range(1, len(rows)+1)
    for row in row_range:
        case_type = driver.find_element_by_xpath('//*[@id="consultationsList"]/div/table/tbody/tr[%s]/td[2]' % row).text
        if case_type == 'E':
            filing_date = driver.find_element_by_xpath('//*[@id="consultationsList"]/div/table/tbody/tr[%s]/td[1]' % row).text
            complaint_status = driver.find_element_by_xpath('//*[@id="consultationsList"]/div/table/tbody/tr[%s]/td[7]' % row).text
            project_link = driver.find_element_by_xpath('//*[@id="consultationsList"]/div/table/tbody/tr[%s]/td[3]/a' % row)
            project = project_link.text
            country = driver.find_element_by_xpath('//*[@id="consultationsList"]/div/table/tbody/tr[%s]/td[4]' % row).text
            year = filing_date[-4:]
            project_link.click()
            time.sleep(1)
            case_text = driver.find_element_by_xpath('//*[@id="consultations"]').text
            junk, case_text = case_text.split('Reference: ', 1)
            case_id, junk = case_text.split('\n', 1)
            try: 
                junk, case_text = case_text.strip().split('Complainant: ', 1)
                filer, junk = case_text.split('\n', 1)
            except ValueError:
                print('Error')
            registration_start_date = filing_date
            try:
                registration_end_date = driver.find_element_by_xpath('//*[@id="consultations"]/div[1]/div[1]/div[2]').text
                eligibility_start_date = registration_end_date
                eligibility_end_date = driver.find_element_by_xpath('//*[@id="consultations"]/div[1]/div[2]/div[2]').text
                cr_start_date = driver.find_element_by_xpath('//*[@id="consultations"]/div[1]/div[3]/div[2]').text
                dr_end_date = driver.find_element_by_xpath('//*[@id="consultations"]/div[1]/div[4]/div[2]').text
                cr_end_date = driver.find_element_by_xpath('//*[@id="consultations"]/div[1]/div[5]/div[2]').text
                date_closed = driver.find_element_by_xpath('//*[@id="consultations"]/div[1]/div[6]/div[2]').text
                monitoring_end_date = driver.find_element_by_xpath('//*[@id="consultations"]/div[1]/div[7]/div[2]').text
            except NoSuchElementException:
                print ('No Date')
            row_data = [
                'EIB',
                year,
                country,
                project,
                case_id,
                29,
                filer,
                None,
                None,
                None,
                None,
                None,
                None,
                None,
                None,
                None,
                complaint_status,
                filing_date,
                registration_start_date,
                registration_end_date,
                eligibility_start_date,
                eligibility_end_date,
                None,
                dr_end_date,
                cr_start_date,
                cr_end_date,
                None,
                monitoring_end_date,
                date_closed,
                None,
                driver.current_url,
            ]
            writer.writerow(row_data)
            print(row_data)
            driver.execute_script('window.history.go(-1)')
            time.sleep(0.7)
            select_all = driver.find_element_by_xpath('//*[@id="consultationsList"]/div/div[3]/div[1]/select/option[5]')
            select_all.click()
            time.sleep(2)
            driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
            time.sleep(1)
            scroll_modifier = int(row) * 40
            driver.execute_script("window.scrollTo(0, %s)" % scroll_modifier) 

In [13]:
scrape()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="consultationsList"]/div/div[3]/div[1]/select/option[5]"}
  (Session info: chrome=66.0.3359.181)
  (Driver info: chromedriver=2.38.552518 (183d19265345f54ce39cbb94cf81ba5f15905011),platform=Mac OS X 10.12.5 x86_64)


---------------------

## Panel Scraper

In [2]:
from builtins import str
from builtins import range
from builtins import object

import time
import unicodecsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

def scrape():
    with open('panel_scraped_new.csv', 'wb') as file:
        writer = unicodecsv.writer(file)
        header = [
            'IAM',
            'Year',
            'Country',
            'Project',
            'ID',
            'IAM ID',
            'Filer(s)',
            'Environmental Category',
            'Project Company',
            'Project Number',
            'Related Project Number',
            'Project Type',
            'Financial Institution',
            'Project Loan Amount',
            'Sector',
            'Issues',
            'Complaint Status',
            'Filing Date',
            'Registration Start Date',
            'Registration End Date',
            'Eligibility Start Date',
            'Eligibility End Date',
            'Dispute Resolution Start Date',
            'Dispute Resolution End Date',
            'Compliance Review Start Date',
            'Compliance Review End Date',
            'Monitoring Start Date',
            'Monitoring End Date',
            'Date Closed',
            'Documents',
            'Hyperlink',
            'Compliance Report Issued?'
        ]
        writer.writerow(header)
        driver = webdriver.Chrome()
        inspection_panel_scrape(driver, writer)
        time.sleep(2)
        driver.quit()

def inspection_panel_scrape(driver, writer):
    driver.get("http://ewebapps.worldbank.org/apps/ip/Pages/AllPanelCases.aspx")
    time.sleep(3)
    rows = driver.find_elements_by_xpath('//*[@id="tblnewAdd"]/tbody/tr')
    row_range = range(2, len(rows)+1)
    for row in row_range:
        # iframe = WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
        # browser.switch_to.frame(iframe)[0]
        case_id = driver.find_element_by_xpath('//*[@id="tblnewAdd"]/tbody/tr[%s]/td[1]' % row).text
        time.sleep(1)
        country = driver.find_element_by_xpath('//*[@id="tblnewAdd"]/tbody/tr[%s]/td[2]' % row).text
        year = driver.find_element_by_xpath('//*[@id="tblnewAdd"]/tbody/tr[%s]/td[3]' % row).text
        project = driver.find_element_by_xpath('//*[@id="tblnewAdd"]/tbody/tr[%s]/td[4]' % row).text 
        link = driver.find_element_by_xpath('//*[@id="tblnewAdd"]/tbody/tr[%s]/td[4]/a' % row)
        link.click()
        time.sleep(1)
        project_id_number = driver.find_element_by_xpath('//*[@id="projectInformationID"]').text
        environmental_category = driver.find_element_by_xpath('//*[@id="projectInformationCategory"]').text
        financial_institution = driver.find_element_by_xpath('//*[@id="projectInformationGrant"]').text
        try: 
            filers = driver.find_element_by_xpath('//*[@id="caseInformationRequesters"]/div/p').text
        except NoSuchElementException:
            filers = driver.find_element_by_id('caseInformationRequesters').text
        stages = driver.find_elements_by_xpath('//*[@id="CaseViewTabs"]/ul/li')
        stage_range = range(1, len(stages)+1)
        for stage in stage_range:
            each_stage = driver.find_element_by_xpath('//*[@id="CaseViewTabs"]/ul/li[%s]' % stage)
            stage_class = each_stage.get_attribute('class')
            stage_text = each_stage.text
            for i in stage_text:
                if stage_text == 'Notice of Receipt':
                    if stage_class in ('completedCurrent', 'completedCurrent active', 'liClassFirst completedCurrent'):
                        date_filed = 'Completed'
                    else: 
                        date_filed = None
                if stage_text == 'Registration of Request':
                    if stage_class in ('completedCurrent', 'completedCurrent active', 'liClassFirst completedCurrent'):
                        registration_start_date = 'Completed'
                        registration_end_date = 'Completed'
                    else: 
                        registration_start_date = None
                        registration_end_date = None
                if stage_text in ('Management Response', 'Eligibility Report'):
                    if stage_class in ('completedCurrent', 'completedCurrent active', 'liClassFirst completedCurrent'):
                        eligibility_start_date = 'Completed'
                    else: 
                        eligibility_start_date = None
                if stage_text == 'Board Approval':
                    if stage_class in ('completedCurrent', 'completedCurrent active', 'liClassFirst completedCurrent'):
                        eligibility_end_date = 'Completed'
                    else: 
                        eligibility_end_date = None
                if stage_text in ('Investigation Ongoing', 'Investigation Report', 'Management Report'):
                    if stage_class in ('completedCurrent', 'completedCurrent active', 'liClassFirst completedCurrent'):
                        cr_start_date = 'Completed'
                    else: 
                        cr_start_date = None
                if stage_text == 'Investigation Report':
                    if stage_class in ('completedCurrent', 'completedCurrent active', 'liClassFirst completedCurrent'):
                        cr_report = 'TRUE'
                    else: 
                        cr_report = 'FALSE'
                if stage_text == 'Board Discussion':
                    if stage_class in ('completedCurrent', 'completedCurrent active', 'liClassFirst completedCurrent'):
                        cr_end_date = 'Completed'
                    else: 
                        cr_end_date = None
                if stage_text == 'Follow up and Progress Report':
                    if stage_class in ('completedCurrent', 'completedCurrent active', 'liClassFirst completedCurrent'):
                        monitoring_start_date = 'Completed'
                        monitoring_end_date = 'Completed'
                    else: 
                        monitoring_start_date = None
                        monitoring_end_date = None
                if stage_text == 'Process Completed':
                    if stage_class in ('completedCurrent', 'completedCurrent active', 'liClassFirst completedCurrent'):
                        date_closed = 'Completed'
                        complaint_status = 'Closed'
                    else: 
                        date_closed = None
                        complaint_status = 'Open'
        documents_list = driver.find_elements_by_class_name('DocsList')
        for document in documents_list:
            attached_documents = document.find_element_by_css_selector('a').get_attribute('href')
        complaint_window = driver.window_handles[0]
        try: 
            project_link = driver.find_element_by_id('projectInformationTitle')
            project_link.click()
            time.sleep(2)
            project_window = driver.window_handles[1]
            driver.switch_to_window(project_window)
            project_details_link = driver.find_element_by_xpath('//*[@id="projdetailsId"]')
            project_details_link.click()
            time.sleep(4)
            try:
                print('we here!')
                project_company = driver.find_element_by_xpath('//*[@id="projectDetails"]/div[1]/div[2]/table/tbody/tr[3]/td[2]').text
                print('did this run?')
            except Exception as error:
                print('are we here?')
                print(error)
                project_company = 'None'
            project_loan = driver.find_element_by_xpath('//*[@id="projectDetails"]/div[1]/div[2]/table/tbody/tr[5]/td[2]').text
            sectors = driver.find_elements_by_xpath('//*[@id="projectDetails"]/div[2]/div[1]/div')
            sector_range = (1, len(sectors))
            sector_list = []
            for i in sector_range:
                sector = driver.find_element_by_xpath('//*[@id="projectDetails"]/div[2]/div[1]/div[%s]/div/div/div[1]' % i).text
                sector_list.append(sector)
            issues = driver.find_elements_by_xpath('//*[@id="accordion_theme"]/div')
            issue_range = range(1, len(issues)+1)
            issue_list = []
            for i in issue_range:
                issue = driver.find_element_by_xpath('//*[@id="accordion_theme"]/div[%s]/div[1]/span' % i).text
                issue_list.append(issue)
            driver.close()
            driver.switch_to_window(complaint_window)
        except Exception as error:
            print (error)
            print ('No project link')
        row_data = [
            'Panel',
            year,
            country,
            project,
            case_id,
            23,
            filers,
            environmental_category,
            project_company,
            project_id_number,
            None,
            None,
            financial_institution,
            project_loan,
            sector_list,
            issue_list,
            complaint_status,
            date_filed,
            registration_start_date,
            registration_end_date,
            eligibility_start_date,
            eligibility_end_date,
            None,
            None,
            cr_start_date,
            cr_end_date,
            monitoring_start_date,
            monitoring_end_date,
            date_closed,
            None,# attached_documents,
            driver.current_url,
            cr_report
        ]
        writer.writerow(row_data)
        print(row_data)
        time.sleep(0.25)
        driver.execute_script('window.history.go(-1)')
        time.sleep(2)
    return True

In [3]:
scrape()

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="projdetailsId"]"}
  (Session info: chrome=67.0.3396.79)
  (Driver info: chromedriver=2.38.552518 (183d19265345f54ce39cbb94cf81ba5f15905011),platform=Mac OS X 10.12.5 x86_64)

No project link


UnboundLocalError: local variable 'project_company' referenced before assignment

---------------

------------

## ADB Scraper


In [1]:
from builtins import str
from builtins import range
from builtins import object

import time
import unicodecsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

def scrape():
    with open('adb_spf_scraped.csv', 'wb') as file:
        writer = unicodecsv.writer(file)
        header = [
            'IAM',
            'Project Name',
            'Project Number',
            'Country',
            'Project Status',
            'Project Financing',
            'Sector',
        ]
        writer.writerow(header)
        driver = webdriver.Chrome()
        adb_spf_scrape(driver, writer)
        time.sleep(2)
        driver.quit()

def adb_spf_scrape(driver, writer):
    driver.get("https://www.adb.org/site/accountability-mechanism/problem-solving-function/complaint-registry-year")
    time.sleep(2)
    tables = driver.find_elements_by_class_name('table-striped')
    table_range = range(1, len(tables)+1)
    for table in table_range:
        rows = driver.find_elements_by_xpath('/html/body/div/div/div/div/main/table[%s]/tbody/tr' % table)
        row_range = range(1, len(rows)+1)
        print(row_range)
        for row in row_range:
            try:
                project_link = driver.find_element_by_xpath('/html/body/div/div/div/div/main/table[%s]/tbody/tr[%s]/td[2]/a' % (table, row))
                time.sleep(2)
                project_id = driver.find_element_by_xpath('/html/body/div/div/div/div/main/table[%s]/tbody/tr[%s]/td[1]' % (table, row))
            except NoSuchElementException:
                project_link = driver.find_element_by_xpath('/html/body/div/div/div/div/main/table[%s]/tbody/tr[%s]/td[2]' % (table, row))
                time.sleep(2)
            try:
                project_link.click()
                time.sleep(1)
            except NoSuchElementException:
                project_name = project_link
                issues = driver.find_element_by_xpath('/html/body/div/div/div/div/main/table[%s]/tbody/tr[%s]/td[3]/ul/li' % (table, row)).text
                project_id = driver.find_element_by_xpath('/html/body/div/div/div/div/main/table[%s]/tbody/tr[%s]/td[1]' % (table, row)).text
            try:
                project_data = driver.find_element_by_xpath('//*[@id="tabs-0"]/li[2]/a')
                project_data.click()
                time.sleep(1)
                project_rows = driver.find_elements_by_xpath('//*[@id="project-pds"]/div/div/div/table[1]/tbody/tr')
                project_row_range = range(1, len(project_rows))
                for project_row in project_row_range:
                    table_div = driver.find_element_by_xpath('//*[@id="project-pds"]/div/div/div/table[1]/tbody/tr[%s]/td[1]' % project_row)
                    if table_div.text == 'Project Name':
                        project_name = driver.find_element_by_xpath('//*[@id="project-pds"]/div/div/div/table[1]/tbody/tr[%s]/td[2]' % project_row).text
                    elif table_div.text == 'Project Number':
                        project_number = driver.find_element_by_xpath('//*[@id="project-pds"]/div/div/div/table[1]/tbody/tr[%s]/td[2]' % project_row).text
                    elif table_div.text == 'Country':
                        country = driver.find_element_by_xpath('//*[@id="project-pds"]/div/div/div/table[1]/tbody/tr[%s]/td[2]' % project_row).text
                    elif table_div.text == 'Project Status':
                        project_status = driver.find_element_by_xpath('//*[@id="project-pds"]/div/div/div/table[1]/tbody/tr[%s]/td[2]' % project_row).text
                    elif table_div.text == 'Project Type / Modality of Assistance':
                        project_finance_type = driver.find_element_by_xpath('//*[@id="project-pds"]/div/div/div/table[1]/tbody/tr[%s]/td[2]' % project_row).text
                    elif table_div.text == 'Sector / Subsector':
                        sector = driver.find_element_by_xpath('//*[@id="project-pds"]/div/div/div/table[1]/tbody/tr[%s]/td[2]/p/strong' % project_row).text
                    elif table_div.text == 'Description':
                        project_description = driver.find_element_by_xpath('//*[@id="project-pds"]/div/div/div/table[1]/tbody/tr[%s]/td[2]' % project_row).text
                    elif table_div.text == 'Source of Funding / Amount':
                        funds = driver.find_elements_by_class_name('colspan')
                        if funds.count(1):
                            fund_name = funds.text
                            fund_amount = driver.find_element_by_class_name('data-th').text
                        elif funds.count(2):
                            for fund in funds:
                                fund_name = fund.text
                        else: 
                            print('No class')
                                # fund_amount = driver.find
            except NoSuchElementException:
                print ('No Project Link')
            row_data = [
                'ADB SPF',
                project_name,
                project_number,
                country,
                project_status,
                project_finance_type,
                sector,
            ]
            writer.writerow(row_data)
            print(row_data)
            time.sleep(0.25)
            driver.execute_script('window.history.go(-2)')
            time.sleep(0.7)
    return True
    

Appears to still be broken - it is getting stuck on this one project instead of jumping to the table that has 2017 projects. 

In [4]:
scrape()

range(1, 8)
No class
['ADB SPF', u'Peshawar Sustainable Bus Rapid Transit Corridor Project', u'48289-001', u'Pakistan', u'Active', u'Technical Assistance', u'Transport']
No class
['ADB SPF', u'National Highway Network Development in Balochistan Project', u'47281-001', u'Pakistan', u'Active', u'Grant\nLoan', u'Transport']
No class
['ADB SPF', u'Ulaanbaatar Urban Services and Ger Areas Development Investment Program - Tranche 1', u'45007-004', u'Mongolia', u'Active', u'Grant\nLoan', u'Energy']
['ADB SPF', u'GEO: Adjaristsqali Hydropower Project', u'47919-014', u'Georgia', u'Active', u'Grant\nLoan', u'Energy']
No class
['ADB SPF', u'SASEC Second Bangladesh\u2013India Electrical Grid Interconnection Project', u'44192-016', u'Bangladesh', u'Active', u'Loan', u'Energy']
No class
['ADB SPF', u'Sustainable Urban Development Investment Program - Tranche 1', u'42417-023', u'Armenia', u'Active', u'Loan', u'Transport']
['ADB SPF', u'Akmola Electricity Distribution Network Modernization and Expansi

KeyboardInterrupt: 

---------------

## CAO Scraper 

In [10]:
from builtins import str
from builtins import range
from builtins import object

import time
import unicodecsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

def scrape():
    with open('cao_scraped.csv', 'wb') as file:
        writer = unicodecsv.writer(file)
        header = [
            'IAM',
            'Year',
            'Country',
            'Project',
            'ID',
            'IAM ID',
            'Filer(s)',
            'Environmental Category',
            'Project Company',
            'Project Number',
            'Related Project Number',
            'Project Type',
            'Financial Institution',
            'Project Loan Amount',
            'Sector',
            'Issues',
            'Complaint Status',
            'Filing Date',
            'Registration Start Date',
            'Registration End Date',
            'Eligibility Start Date',
            'Eligibility End Date',
            'Dispute Resolution Start Date',
            'Dispute Resolution End Date',
            'Compliance Review Start Date',
            'Compliance Review End Date',
            'Monitoring Start Date',
            'Monitoring End Date',
            'Date Closed',
            'Documents',
            'Hyperlink'
        ]
        writer.writerow(header)
        driver = webdriver.Chrome()
        cao_scrape(driver, writer)
        time.sleep(2)
        driver.quit()

def cao_scrape(driver, writer):
    driver.get('http://www.cao-ombudsman.org/cases/default.aspx')
    time.sleep(2)
    button = driver.find_element_by_id('ctl00_MainContent_btnSearch').click()
    time.sleep(2)
    tables = driver.find_elements_by_xpath('//*[@id="ctl00_MainContent_divResults"]/ul')
    table_range = range(1, len(tables)+1)
    for table in table_range:
        rows = driver.find_elements_by_xpath('//*[@id="ctl00_MainContent_divResults"]/ul[%s]/li' % table)
        row_range = range(1, len(rows)+1)
        for row in row_range:
            functions = driver.find_elements_by_xpath('//*[@id="ctl00_MainContent_divResults"]/ul[%s]/li[%s]/ul/li' % (table, row))
            function_range = range(1, len(functions)+1)
            for function in function_range:
                complaints = driver.find_elements_by_xpath('//*[@id="ctl00_MainContent_divResults"]/ul[%s]/li[%s]/ul/li[%s]/ul/li' % (table, row, function))
                complaint_range = range(1, len(complaints)+1)
                for complaint in complaint_range:
                    link = driver.find_element_by_xpath('//*[@id="ctl00_MainContent_divResults"]/ul[%s]/li[%s]/ul/li[%s]/ul/li[%s]/a' % (table, row, function, complaint))
                    link.click()
                    time.sleep(2)
                    project_and_id = driver.find_element_by_id('ctl00_MainContent_ctrlProjectName').text
                    project, project_id = project_and_id.rsplit(' ', 1)
                    country = driver.find_element_by_id('ctl00_MainContent_ctrlProjectCountries').text
                    year = driver.find_element_by_id('ctl00_MainContent_ctrlDateFilled').text
                    year = year[-4:]
                    url = driver.current_url
                    case_id = ''.join([i for i in url if i.isdigit()])
                    environmental_category = driver.find_element_by_id('ctl00_MainContent_ctrlEnvironmentCategory').text
                    project_type = driver.find_element_by_id('ctl00_MainContent_ctrlDepartment').text
                    sector = driver.find_element_by_id('ctl00_MainContent_ctrlSector').text
                    financial_institution = driver.find_element_by_id('ctl00_MainContent_ctrlInstitution').text
                    project_company = driver.find_element_by_id('ctl00_MainContent_ctrlCompany').text
                    project_loan = driver.find_element_by_id('ctl00_MainContent_ctrlCommitment').text
                    issues = driver.find_element_by_id('ctl00_MainContent_ctrlConcerns').text
                    case_status = driver.find_element_by_id('ctl00_MainContent_ctrlCaseStatus').text
                    try: 
                        completed_stage = driver.find_elements_by_class_name('completed')
                        last_completed_stage = completed_stage[-1].text
                    except IndexError: 
                        closed_stage = driver.find_element_by_class_name('closed')
                        last_completed_stage = closed_stage.text
                    try: 
                        active_stage = driver.find_element_by_class_name('inprocess')
                    except NoSuchElementException:
                        active_stage = None
                    if active_stage in ('Eligibility: In Process', 'Eligible: In Process'):
                        eligibility_start_date = 'Completed'
                    else: 
                        eligibility_start_date = None
                    if last_completed_stage in ('Eligibility: Completed', 'Eligible: Completed'):
                        eligibility_end_date = 'Completed'
                    else: 
                        eligibility_end_date = None
                    if active_stage in ('Assessment Period: In Process', 'Facilitating Settlement: Active'):
                        dr_start_date = 'Completed'
                    elif last_completed_stage == 'Assessment Period: Completed':
                        dr_start_date = 'Completed'
                    else: 
                        dr_start_date = None 
                    if last_completed_stage == 'Facilitating Settlement: Completed':
                        dr_end_date = 'Completed'
                    else: 
                        dr_end_date = None
                    if active_stage in ('Monitoring/Close Out: In Process', 'Monitoring: In Process'):
                        monitoring_start_date = 'Completed'
                    else: 
                        monitoring_start_date = None    
                    if last_completed_stage == 'Monitoring/Close Out: Completed':
                        monitoring_end_date = 'Completed'
                    else: 
                        monitoring_end_date = None
                    if active_stage in ('Under Appraisal: In Process', 'Under Audit: In Process'):
                        cr_start_date = 'Completed'
                    elif last_completed_stage == 'Under Appraisal: Completed':
                        cr_start_date = 'Completed'
                    else: 
                        cr_start_date = None
                    if last_completed_stage == 'Under Audit: Compelted':
                        cr_end_date = 'Completed'
                    else: 
                        cr_end_date = None
                    if last_completed_stage == 'Monitoring: Completed':
                        date_closed = 'Completed'
                    else: 
                        date_closed = None
                    date_filed = driver.find_element_by_id('ctl00_MainContent_ctrlDateFilled').text
                    registration_start_date = date_filed
                    registration_end_date = None
                    row_data = [
                        'CAO',
                        year,
                        country,
                        project,
                        case_id,
                        21,
                        None,
                        environmental_category,
                        project_company,
                        project_id,
                        None,
                        project_type,
                        financial_institution,
                        project_loan,
                        sector,
                        issues,
                        case_status,
                        date_filed,
                        registration_start_date,
                        registration_end_date,
                        eligibility_start_date,
                        eligibility_end_date,
                        dr_start_date,
                        dr_end_date,
                        cr_start_date,
                        cr_end_date,
                        None,
                        monitoring_end_date,
                        date_closed,
                        None,
                        driver.current_url,
                    ]
                    writer.writerow(row_data)
                    print(row_data)
                    driver.execute_script('window.history.go(-1)')
                    time.sleep(0.7)
    return True


In [11]:
scrape()

['CAO', u'2009', u'Cambodia', u'Cambodia Airport II', u'155', 21, None, u'B', u"Societe Concessionaire de l'Aeroport", u'25332', None, u'Infrastructure', u'IFC', u'$7.5 million', u'Transportation', u'Land acquisition and compensation, loss of livelihoods, noise pollution, environmental impact to a national park, categorization, community consultation, and disclosure of project information', u'Closed', u'December 11, 2009', u'December 11, 2009', None, None, None, None, None, None, None, None, None, None, None, u'http://www.cao-ombudsman.org/cases/case_detail.aspx?id=155']
['CAO', u'2013', u'Cambodia', u'Cambodia Airports', u'205', 21, None, u'B', u"Societe Concessionaire de l'Aeroport", u'21363', None, u'Infrastructure', u'IFC', u'$10 million', u'Transportation', u'Land acquisition, forced eviction, community consultation and due diligence', u'Open - Ombudsman', u'June 07, 2013', u'June 07, 2013', None, None, None, None, 'Completed', None, None, None, None, None, None, u'http://www.cao-

KeyboardInterrupt: 

----------------

## Mici Scrape

In [8]:
from builtins import str
from builtins import range
from builtins import object

import time
import unicodecsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

def scrape():
    with open('mici_scraped.csv', 'wb') as file:
        writer = unicodecsv.writer(file)
        header = [
            'IAM',
            'Year',
            'Country',
            'Project',
            'ID',
            'IAM ID',
            'Filer(s)',
            'Environmental Category',
            'Project Company',
            'Project Number',
            'Related Project Number',
            'Project Type',
            'Financial Institution',
            'Project Loan Amount',
            'Sector',
            'Issues',
            'Complaint Status',
            'Filing Date',
            'Registration Start Date',
            'Registration End Date',
            'Eligibility Start Date',
            'Eligibility End Date',
            'Dispute Resolution Start Date',
            'Dispute Resolution End Date',
            'Compliance Review Start Date',
            'Compliance Review End Date',
            'Monitoring Start Date',
            'Monitoring End Date',
            'Date Closed',
            'Documents',
            'Hyperlink',
        ]
        writer.writerow(header)
        driver = webdriver.Chrome()
        mici_scrape(driver, writer)
        time.sleep(2)
        driver.quit()

def mici_scrape(driver, writer):
    driver.get("https://idblegacy.iadb.org/en/mici/by-country,7631.html")
    time.sleep(2)
    page_range = range(2,14)
    rows = driver.find_elements_by_xpath('//*[@id="searchResults"]/li')
    for page in page_range:
        row_range = range(1, len(rows)+1)
        for row in row_range:
            project_and_id = driver.find_element_by_xpath('//*[@id="searchResults"]/li[%s]/h5/a' % row).text
            try:
                project, case_id = project_and_id.rsplit(' (', 1)
            except:
                project = None
                case_id = project_and_id
            case_id = case_id.replace(')','')
            country_and_year = driver.find_element_by_xpath('//*[@id="searchResults"]/li[%s]/div/span[1]' % row).text
            country, year = country_and_year.split(' - ')
            project_link = driver.find_element_by_xpath('//*[@id="searchResults"]/li[%s]/h5/a' % row)
            project_link.click()
            time.sleep(2)
            try:
                complaint_status_string = driver.find_element_by_xpath('//*[@id="container_2"]/div/div/div/div[2]/table/tbody/tr[2]/td[1]/p').text 
                junk, complaint_status = complaint_status_string.rsplit(':  ', 1)
            except ValueError:
                url = driver.current_url
                print('project blank %s' %url)
            try: 
                filers = driver.find_element_by_xpath('//*[@id="container_2"]/div/div/div/div[2]/table/tbody/tr[3]/td/div[2]/div/p[2]').text
            except NoSuchElementException:
                filers = None
            try:
                for i in range(1, 8):
                    header = driver.find_element_by_xpath('//*[@id="container_2"]/div/div/div/div[3]/div/b[%s]' % i)
                    if header.text == 'Project Number:':
                        project_number_xpath = '//*[@id="container_2"]/div/div/div/div[3]/div/p[%s]' % i
                        project_number = driver.find_element_by_xpath(project_number_xpath).text
                    elif header.text == 'Other related projects:':
                        related_project_xpath = '//*[@id="container_2"]/div/div/div/div[3]/div/p[%s]' % i
                        related_project_number = driver.find_element_by_xpath(related_project_xpath).text
                    elif header.text == 'Environmental Category:':
                        environmental_category_xpath = '//*[@id="container_2"]/div/div/div/div[3]/div/p[%s]' % i
                        environmental_category = driver.find_element_by_xpath(environmental_category_xpath).text
                    elif header.text == 'Project Name:':
                        project_name_xpath = '//*[@id="container_2"]/div/div/div/div[3]/div/p[%s]' % i
                        project_name = driver.find_element_by_xpath(project_name_xpath).text
                    elif header.text == 'Sector:':
                        sector_xpath = '//*[@id="container_2"]/div/div/div/div[3]/div/p[%s]' % i
                        sector = driver.find_element_by_xpath(sector_xpath).text
                    elif header.text == 'Project Type:':
                        project_type_xpath = '//*[@id="container_2"]/div/div/div/div[3]/div/p[%s]' % i
                        project_type = driver.find_element_by_xpath(project_type_xpath).text
                    elif header.text == 'IDB Financing:':
                        idb_financing_xpath = '//*[@id="container_2"]/div/div/div/div[3]/div/p[%s]' % i
                        financing = driver.find_element_by_xpath(idb_financing_xpath).text
            except NoSuchElementException: 
                related_project_number = None
            if year > '2014':
                try:
                    completed_reg = driver.find_elements_by_class_name('sltgray')
                    last_completed_reg = completed_reg[-1].text.strip()
                    print(last_completed_reg)
                except IndexError:
                    last_completed_reg = None
                    print('No Registration Undertaken')
                if last_completed_reg in ('Registration Not registered'):
                    registration_start_date = 'Completed'
                    registration_end_date = None
                else: 
                    registration_start_date = None
                    registration_end_date = None
                if last_completed_reg in ('Eligibility Eligible', 'Eligibility', 'Eligibility Not eligible'):
                    eligibility_start_date = 'Completed'
                    eligibility_end_date = 'Completed'
                else:
                    eligibility_start_date = None
                    eligibility_end_date = None
                try:
                    completed_dr = driver.find_elements_by_class_name('sltgreen')
                    last_completed_dr = completed_dr[-1].text.strip()
                    print(last_completed_dr)
                except IndexError:
                    last_completed_dr = None
                    print('No Dispute Resolution Undertaken')
                if last_completed_dr in ('Assessment', 'Assessment Transferred'):
                    dr_start_date = 'Completed'
                else: 
                    dr_start_date = None
                if last_completed_dr == 'Consultation Process':
                    dr_end_date = 'Completed'
                else: 
                    dr_end_date = None
                if last_completed_dr == 'Monitoring':
                    if complaint_status == 'Open':
                        monitoring_start_date = 'Completed'
                    elif complaint_status == 'Closd':
                        monitoring_end_date = 'Completed'
                        date_closed = 'Completed'
                else: 
                    monitoring_start_date = None
                    monitoring_end_date = None
                    date_closed = None
                try: 
                    completed_cr = driver.find_elements_by_class_name('cuadro_dw_azul')
                    last_completed_cr = completed_cr[-1].text.strip()
                    print(last_completed_cr)
                except IndexError:
                    last_completed_cr = None
                    print('No Compliance Review Undertaken')
                if last_completed_cr == 'Monitoring':
                    if complaint_status == 'Open':
                        monitoring_start_date = 'Completed'
                    elif complaint_status == 'Closed':
                        monitoring_end_date = 'Completed'
                        date_closed = 'Completed'
                else: 
                    monitoring_start_date = None
                    monitoring_end_date = None  
                    date_closed = None
                if last_completed_cr in ('Recommendation for a CR and ToRs', 'Investigation'):
                    cr_start_date = 'Completed'
                else: 
                    cr_start_date = None
                if last_completed_cr in ('CR Report', 'Recommendation for a CR and ToRs Closed', 'Investigation Closed'):
                    cr_end_date = 'Completed'
                else: 
                    cr_end_date = None
            elif year <= '2014':
                stages = driver.find_elements_by_xpath('//*[@id="container_2"]/div/div/div/div[4]/div/div[1]/div')
                stage_range = range(1, len(stages)+1)
                for stage in stage_range:
                    try: 
                        completed_dr = driver.find_elements_by_class_name('cuadro_dw')
                        last_completed_dr = completed_dr[-1].text.strip()
                    except IndexError:
                        last_completed_dr = None
                    try: 
                        completed_cr = driver.find_elements_by_class_name('cuadro_dw_naranja')
                        last_completed_cr = completed_cr[-1].text.strip()
                    except IndexError:
                        last_completed_cr = None
                    if last_completed_dr == 'Eligibility':
                        eligibility_end_date = 'Completed'
                    elif last_completed_cr == 'Eligibility':
                        eligibility_end_date = 'Completed'
                    else:
                        eligibility_end_date = None
                    if last_completed_dr == 'Assessment':
                        dr_start_date = 'Completed'
                    else:
                        dr_start_date = None
                    if last_completed_dr == 'Consultation Phase Exercise':
                        dr_end_date = 'Completed'
                    else:
                        dr_end_date = None
                    if last_completed_dr == 'Monitoring':
                        monitoring_end_date = 'Completed'
                    elif last_completed_cr == 'Monitoring':
                        monitoring_end_date = 'Completed'
                    else:
                        monitoring_end_date = None
                    if last_completed_cr in ('Preparation of TORs', 'Investigation'):
                        cr_start_date = 'Completed'
                    else:
                        cr_start_date = None
                    if last_completed_cr == 'Panel Report':
                        cr_end_date = 'Completed'
                    else:
                        cr_end_date = None
            row_data = [
                'MICI',
                year,
                country,
                project_name,
                case_id,
                24,
                filers,
                environmental_category,
                None,
                project_number,
                related_project_number,
                project_type,
                None,
                financing,
                sector,
                None,
                complaint_status,
                None,
                registration_start_date,
                registration_end_date,
                eligibility_start_date,
                eligibility_end_date,
                dr_start_date,
                dr_end_date,
                cr_start_date,
                cr_end_date,
                monitoring_start_date,
                monitoring_end_date,
                date_closed,
                None,
                driver.current_url,
            ]
            writer.writerow(row_data)
            print(row_data)
            time.sleep(0.25)
            driver.execute_script('window.history.go(-1)')
            time.sleep(0.7)
        driver.get('https://idblegacy.iadb.org/en/mici/by-country,7631.html?status=&country=url.country&yearDate=&page=%s' % page)
        time.sleep(2)
    return True

In [9]:
scrape()

Registration
Not registered
No Dispute Resolution Undertaken
No Compliance Review Undertaken
['MICI', u'2018', u'Brazil', u'S\xe3o Jos\xe9 dos Campos Urban Structuring Program', u'MICI-BID-BR-2018-0132', 24, u'A resident of Sao Jose dos Campos, Brazil.', u'B', None, u'BR-L1160', None, u'Loan Operation', None, u'USD 85,672,400', u'URBAN DEVELOPMENT AND HOUSING', None, u'Closed', None, None, None, None, None, None, None, None, None, None, None, None, None, u'https://idblegacy.iadb.org/en/mici/complaint-detail,19172.html?ID=MICI-BID-BR-2018-0132']
Registration
In Process
No Dispute Resolution Undertaken
No Compliance Review Undertaken
['MICI', u'2018', u'Ecuador', u'Program for the Reconstruction of Electricity Infrastructure in Areas Affected by the Earthquake in Ecuador', u'MICI-BID-EC-2018-0131', 24, u'Residents of 8 communities in the province of Manab\xed, Ecuador.', u'B', None, u'EC-L1219', None, u'Loan Operation', None, u'USD 60,000,000', u'ENERGY', None, u'Open', None, None, None,

KeyboardInterrupt: 

# End 